In [1]:
!pip uninstall -qqy jupyterlab  
!pip install -U -q "google-genai==1.7.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.7/144.7 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.9/100.9 kB 4.8 MB/s eta 0:00:00


In [2]:
from google import genai
from google.genai import types

genai.__version__

'1.7.0'

In [3]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

In [4]:
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)
for model in client.models.list():
    if "createTunedModel" in model.supported_actions:
        print(model.name)

models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash-001-tuning


In [7]:
from sklearn.datasets import fetch_20newsgroups

newsgroups_train = fetch_20newsgroups(subset="train")
newsgroups_test = fetch_20newsgroups(subset="test")

print(newsgroups_train.target_names)
print(newsgroups_train.data[0])

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have o

In [9]:
import email
import re
import pandas as pd
def preprocess_newsgroup_row(data):
    msg = email.message_from_string(data)
    text = f"{msg['Subject']}\n\n{msg.get_payload()}"
    text = re.sub(r"[\w\.-]+@[\w\.-]+", "", text)
    text = text[:40000]
    return text
def preprocess_newsgroup_data(newsgroup_dataset):
    df = pd.DataFrame(
        {"Text": newsgroup_dataset.data, "Label": newsgroup_dataset.target}
    )
    df["Text"] = df["Text"].apply(preprocess_newsgroup_row)
    df["Class Name"] = df["Label"].map(lambda l: newsgroup_dataset.target_names[l])
    return df
df_train = preprocess_newsgroup_data(newsgroups_train)
df_test = preprocess_newsgroup_data(newsgroups_test)
df_train.head()

,Text,Label,Class Name
0,WHAT car is this!?\n\n I was wondering if anyo...,7,rec.autos
1,SI Clock Poll - Final Call\n\nA fair number of...,4,comp.sys.mac.hardware
2,"PB questions...\n\nwell folks, my mac plus fin...",4,comp.sys.mac.hardware
3,Re: Weitek P9000 ?\n\nRobert J.C. Kyanko () wr...,1,comp.graphics
4,Re: Shuttle Launch Question\n\nFrom article <>...,14,sci.space


In [13]:
def sample_data(df, num_samples, classes_to_keep):
    df = (
        df.groupby("Label")[df.columns]
        .apply(lambda x: x.sample(num_samples))
        .reset_index(drop=True)
    )
    df = df[df["Class Name"].str.contains(classes_to_keep)]
    df["Class Name"] = df["Class Name"].astype("category")
    return df
TRAIN_NUM_SAMPLES = 50
TEST_NUM_SAMPLES = 10
CLASSES_TO_KEEP = "^rec|^sci"
df_train = sample_data(df_train, TRAIN_NUM_SAMPLES, CLASSES_TO_KEEP)
df_test = sample_data(df_test, TEST_NUM_SAMPLES, CLASSES_TO_KEEP)

In [14]:
#Evaluate baseline performance¶
sample_idx = 0
sample_row = preprocess_newsgroup_row(newsgroups_test.data[sample_idx])
sample_label = newsgroups_test.target_names[newsgroups_test.target[sample_idx]]
print(sample_row)
print('---')
print('Label:', sample_label)

Need info on 88-89 Bonneville


 I am a little confused on all of the models of the 88-89 bonnevilles.
I have heard of the LE SE LSE SSE SSEI. Could someone tell me the
differences are far as features or performance. I am also curious to
know what the book value is for prefereably the 89 model. And how much
less than book value can you usually get them for. In other words how
much are they in demand this time of year. I have heard that the mid-spring
early summer is the best time to buy.

			Neil Gandler

---
Label: rec.autos


In [15]:
response = client.models.generate_content(
    model="gemini-1.5-flash-001", contents=sample_row)
print(response.text)

You're right, the 1988-89 Bonneville trim levels can be confusing! Here's a breakdown and some information on their value:

**1988-1989 Bonneville Trim Levels**

* **LE (Luxury Edition):**  The base model, featuring standard amenities like air conditioning, power steering, and power brakes. It usually came with a 3.8L V6 engine.
* **SE (Special Edition):**  This model included upgraded features like a more luxurious interior, additional sound insulation, and a more powerful 3.8L V6 engine with a higher horsepower rating.
* **LSE (Luxury Special Edition):**  This trim level combined the features of the LE and SE, offering a more upscale interior and a more powerful engine. 
* **SSE (Sport Sedan Edition):**  The SSE was a sportier version with a unique grille, body-colored bumpers, and a more powerful 3.8L V6 engine.
* **SSEi (Sport Sedan Edition - Injection):**  The SSEi was the top-of-the-line model and came with a fuel-injected 3.8L V6 engine for increased power and performance. It al

In [16]:
prompt = "From what newsgroup does the following message originate?"
baseline_response = client.models.generate_content(
    model="gemini-1.5-flash-001",
    contents=[prompt, sample_row])
print(baseline_response.text)

The message most likely originates from a **Buick-specific newsgroup**, potentially one focused on the Bonneville or even the 1988-1989 model years.

Here's why:

* **Specific Model Focus:** The message centers around the Bonneville, a Buick model, and its various trim levels. This strongly suggests a Buick-centric discussion space.
* **Trim Level Questions:** The user inquires about various trim levels (LE, SE, LSE, SSE, SSEi) common to Buick models. This detail further reinforces the Buick context. 
* **Value and Market Information:** The message asks about book value and negotiation tactics specific to the 1989 Bonneville, suggesting an interest in buying and selling within a community familiar with this model.

While it's impossible to pinpoint the exact newsgroup without more information, the clues point to a Buick-specific forum or newsgroup.



In [ ]:
from google.api_core import retry

system_instruct = """
You are a classification service. You will be passed input that represents
a newsgroup post and you must respond with the newsgroup from which the post
originates.
"""

# a helper to retry when per-minute quota is reached.
is_retriable = lambda e: (isinstance(e, genai.errors.APIError) and e.code in {429, 503})

# If you want to evaluate your own technique, replace this body of this function
# with your model, prompt and other code and return the predicted answer.
@retry.Retry(predicate=is_retriable)
def predict_label(post: str) -> str:
    response = client.models.generate_content(
        model="gemini-1.5-flash-001",
        config=types.GenerateContentConfig(
            system_instruction=system_instruct),
        contents=post)

    rc = response.candidates[0]

    # Any errors, filters, recitation, etc we can mark as a general error
    if rc.finish_reason.name != "STOP":
        return "(error)"
    else:
        return response.text.strip()
        
prediction = predict_label(sample_row)

print(prediction)
print()
print("Correct!" if prediction == sample_label else "Incorrect.")

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_train["Text"])
y_train = df_train["Class Name"]

clf = MultinomialNB()
clf.fit(X_train, y_train)

X_test = vectorizer.transform(df_test["Text"])
df_test["Prediction"] = clf.predict(X_test)

accuracy = (df_test["Class Name"] == df_test["Prediction"]).mean()
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 81.25%


In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

#Text vectorization
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_train["Text"])
X_test = vectorizer.transform(df_test["Text"])

clf = RandomForestClassifier()
clf.fit(X_train, df_train["Class Name"])

df_test["Prediction"] = clf.predict(X_test)
accuracy = (df_test["Class Name"] == df_test["Prediction"]).mean()
print(f"TF-IDF Accuracy: {accuracy:.2%}")


TF-IDF Accuracy: 78.75%


In [47]:
df_baseline_eval

,Text,Label,Class Name,Prediction,labels
0,Re: Lexus and Infiniti\n\nIn article <> (Issa...,7,rec.autos,rec.autos,0
1,Re: Is car saftey important?\n\nIn article <> ...,7,rec.autos,rec.autos,0
2,"Re: Back Breaker, Near Hit!! \n\nIn article <1...",8,rec.motorcycles,rec.autos,1
3,Re: Need advice for riding with someone on pil...,8,rec.motorcycles,rec.motorcycles,1
4,Re: Ottawa Lynx info wanted\n\n (CCHB) writes:...,9,rec.sport.baseball,rec.sport.baseball,2
5,Re: Braves Update!!\n\nIn <> (Randy Palermo) ...,9,rec.sport.baseball,rec.sport.baseball,2
6,"Some Play-offs Thoughts\n\n\nWell, we're almos...",10,rec.sport.hockey,rec.sport.hockey,3
7,Re: Wings News and Playoff Thoughts\n\nIn arti...,10,rec.sport.hockey,rec.autos,3
8,Re: Let's build software cryptophones for over...,11,sci.crypt,rec.autos,4
9,Re: Clipper Crap\n\n (Karl Kluge) writes:\n\n>...,11,sci.crypt,sci.crypt,4


In [48]:
#Tuning a custom model
from collections.abc import Iterable
import random

# Convert the data frame into a dataset suitable for tuning.
input_data = {'examples': 
    df_train[['Text', 'Class Name']]
      .rename(columns={'Text': 'textInput', 'Class Name': 'output'})
      .to_dict(orient='records')
 }

#if re-running
model_id = None

if not model_id:
  queued_model = None
  for m in reversed(client.tunings.list()):
    if m.name.startswith('tunedModels/newsgroup-classification-model'):
      if m.state.name == 'JOB_STATE_SUCCEEDED':
        model_id = m.name
        print('Found existing tuned model to reuse.')
        break

      elif m.state.name == 'JOB_STATE_RUNNING' and not queued_model:
        queued_model = m.name
  else:
    if queued_model:
      model_id = queued_model
      print('Found queued model, still waiting.')

if not model_id:
    tuning_op = client.tunings.tune(
        base_model="models/gemini-1.5-flash-001-tuning",
        training_dataset=input_data,
        config=types.CreateTuningJobConfig(
            tuned_model_display_name="Newsgroup classification model",
            batch_size=16,
            epoch_count=2,
        ),
    )
    print(tuning_op.state)
    model_id = tuning_op.name
print(model_id)

<ipython-input-48-3f54e37193ba>:32: ExperimentalWarning: The SDK's tuning implementation is experimental, and may change in future versions.
  tuning_op = client.tunings.tune(


JobState.JOB_STATE_QUEUED
tunedModels/newsgroup-classification-model-k5u8e1ssb


In [49]:
import datetime
import time

MAX_WAIT = datetime.timedelta(minutes=10)

while not (tuned_model := client.tunings.get(name=model_id)).has_ended:

    print(tuned_model.state)
    time.sleep(60)

    if datetime.datetime.now(datetime.timezone.utc) - tuned_model.create_time > MAX_WAIT:
        print("Taking a shortcut, using a previously prepared model.")
        model_id = "tunedModels/newsgroup-classification-model-ltenbi1b"
        tuned_model = client.tunings.get(name=model_id)
        break
print(f"Done! The model state is: {tuned_model.state.name}")

if not tuned_model.has_succeeded and tuned_model.error:
    print("Error:", tuned_model.error)

JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
JobState.JOB_STATE_RUNNING
Taking a shortcut, using a previously prepared model.
Done! The model state is: JOB_STATE_SUCCEEDED


In [52]:
new_text = """
First-timer looking to get out of here.

Hi, I'm writing about my interest in travelling to the outer limits!

What kind of craft can I buy? What is easiest to access from this 3rd rock?

Let me know how to do that please.
"""

response = client.models.generate_content(
    model=model_id, contents=new_text)

print(response.text)

sci.space


In [68]:
import time
from google.api_core.exceptions import ResourceExhausted
from tenacity import retry, stop_after_attempt, wait_exponential

@retry(wait=wait_exponential(multiplier=2, min=5, max=60), stop=stop_after_attempt(5))
def classify_text(text: str) -> str:
    """Classify the provided text into a known newsgroup with retries."""
    try:
        response = client.models.generate_content(model=model_id, contents=text)
        rc = response.candidates[0]

        if rc.finish_reason.name != "STOP":
            return "(error)"
        else:
            return rc.content.parts[0].text

    except ResourceExhausted:
        print("Quota exceeded. Retrying...")
        raise  

In [64]:
system_instruct = "You are an AI assistant that helps classify text into categories."
sample_row = "This is a test sentence for classification."

sysint_tokens = client.models.count_tokens(
    model='gemini-1.5-flash-001', contents=[system_instruct, sample_row]
).total_tokens
print(f'System instructed baseline model: {sysint_tokens} (input)')

# Calculate the input cost of the tuned model.
tuned_tokens = client.models.count_tokens(model=tuned_model.base_model, contents=sample_row).total_tokens
print(f'Tuned model: {tuned_tokens} (input)')

savings = (sysint_tokens - tuned_tokens) / tuned_tokens
print(f'Token savings: {savings:.2%}')  # Note that this is only n=1.

System instructed baseline model: 21 (input)
Tuned model: 9 (input)
Token savings: 133.33%
